<a href="https://colab.research.google.com/gist/parulnith/7f8c174e6ac099e86f0495d3d9a4c01e/untitled9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music genre classification notebook

## Importing Libraries

In [38]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import librosa.display
import pathlib
import csv


# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

## Extracting music and features

### Dataset

We use [GTZAN genre collection](http://marsyasweb.appspot.com/download/data_sets/) dataset for classification. 
<br>
<br>
The dataset consists of 10 genres i.e
 * Blues
 * Classical
 * Country
 * Disco
 * Hiphop
 * Jazz
 * Metal
 * Pop
 * Reggae
 * Rock
 
Each genre contains 100 songs. Total dataset: 1000 songs

In [5]:
!wget https://www.dropbox.com/s/yqg3c9mdmq665fy/genres.zip

--2020-12-16 17:46:15--  https://www.dropbox.com/s/yqg3c9mdmq665fy/genres.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/yqg3c9mdmq665fy/genres.zip [following]
--2020-12-16 17:46:16--  https://www.dropbox.com/s/raw/yqg3c9mdmq665fy/genres.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce5b7379de90588280228d1c637.dl.dropboxusercontent.com/cd/0/inline/BFOd0cST_v0IaZbsyGoJ9glwBXaNz-_822siMvJ4eWeWm8hPvZigkL--TxSGV8kNeotwHtFMx4be4Fv4qWcbtvLqIu1hlUIuGnDnTadVP2G5F7roH9VPl-viJW1vLWtcHMQ/file# [following]
--2020-12-16 17:46:16--  https://uce5b7379de90588280228d1c637.dl.dropboxusercontent.com/cd/0/inline/BFOd0cST_v0IaZbsyGoJ9glwBXaNz-_822siMvJ4eWeWm8hPvZigkL--TxSGV8kNeotwHtFMx4be4Fv4qWcbtvLqIu1hlUIuGnDnTadVP2G5F7roH9VPl

In [ ]:
!unzip '/content/genres.zip'

In [7]:
!rm '/content/genres.zip'

## Extracting the Spectrogram for every Audio

In [ ]:
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'/content/output/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'/content/genres/{g}'):
        #print (filename)
        #print ({g})
        songname = f'/content/genres/{g}/{filename}'
        #print (songname)
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'/content/output/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

In [ ]:
genres = 'blues classical'.split()
for g in genres:
  for filename in os.listdir(f'/content/genres/{g}'):       
      songname = f'/content/genres/{g}/{filename}'
      X = librosa.stft(y) #converts to short term fourier transform
      Xdb = librosa.amplitude_to_db(abs(X))
      plt.figure(figsize=(14, 5))
      librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz') 
      plt.colorbar()

All the audio files get converted into their respective spectrograms which is a visual representation of the spectrum of frequencies of sound or other signals as they vary with time. Above are shown for blues and classical only.

## Extracting features from Spectrogram


We will extract

* Mel-frequency cepstral coefficients (MFCC)(20 in number)
* Spectral Centroid,
* Zero Crossing Rate
* Chroma Frequencies
* Spectral Roll-off.

In [42]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

## Writing data to csv file

We write the data to a csv file 

In [45]:
file = open('data.csv', 'w', newline='') #data.csv in write mode
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'/content/genres/{g}'):
        songname = f'/content/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        rmse = librosa.feature.rmse(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='') #data.csv in append mode
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

The data has been extracted into a [data.csv](https://github.com/parulnith/Music-Genre-Classification-with-Python/blob/master/data.csv) file.

# Analysing the Data in Pandas

In [46]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00030.au,0.263016,0.170081,1379.081742,2004.000850,3015.831764,0.039376,-206.987590,117.781468,23.256245,28.294068,-1.099598,18.864939,-21.953711,17.855118,-25.583357,2.128552,-17.789054,-8.015467,-17.616342,-8.138554,-8.646157,-15.538988,-15.331506,-9.664872,-10.103310,-17.835100,blues
1,blues.00073.au,0.341865,0.259714,2094.919420,2158.335963,4482.447895,0.100570,-30.094571,103.732289,-19.754240,56.404770,-12.710990,20.188002,-17.340381,16.292435,-18.439600,8.924175,-11.648016,11.213472,-8.842511,9.090302,-3.579252,4.372714,-2.830807,4.725089,0.485058,1.475955,blues
2,blues.00044.au,0.390212,0.136276,2279.124558,2375.102120,5198.360233,0.092570,-109.509285,86.922406,-8.607986,64.494557,-6.304126,43.639515,-8.286753,28.546109,-14.079793,8.125040,-21.491401,11.087480,-5.085794,3.976359,-12.859741,12.343857,0.026217,-0.741570,-5.126619,3.303441,blues
3,blues.00037.au,0.248627,0.069145,1188.168337,1682.860150,2339.635853,0.048160,-328.673793,102.696873,19.876064,26.731257,7.158258,22.716643,-6.956797,12.804352,-10.708748,-0.419349,-12.363976,-8.897177,-9.238519,-9.109326,-9.003836,-7.640767,-7.190916,-9.368477,-4.470060,-3.688800,blues
4,blues.00008.au,0.409037,0.142438,1719.213163,2031.643884,3781.318802,0.064028,-121.361024,122.513116,-14.742130,46.143444,-8.165334,20.176526,-19.172478,23.055617,-11.830503,21.177009,-6.721190,7.010945,-12.741833,5.066005,-5.175478,-1.444025,-7.037070,2.697451,3.409809,-2.698353,blues


In [47]:
data.shape

(1000, 28)

In [48]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)

## Encoding the Labels

In [49]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

## Scaling the Feature columns using Standard Scaler

In [50]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

## Dividing data into training and Testing set

In [142]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [143]:
len(y_train)

800

In [144]:
len(y_test)

200

In [145]:
X_train[10]

array([ 1.57084109, -0.33460798,  1.22578333,  1.85963298,  1.54478988,
        0.08228682, -0.00470943, -1.04999786,  2.45358366, -0.82609452,
        0.62790485, -0.48863984,  1.71322289, -0.40956386,  1.21272377,
       -0.16079748,  1.40641243, -0.29009228,  1.39898036,  0.3873688 ,
        2.1949732 ,  0.85717042,  0.97770587,  0.14887718,  1.14075537,
        0.26495529])

# Classification with Keras

## Building our Network

In [160]:
from keras import models
from keras import layers
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

In [147]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [148]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)
                   

Epoch 1/20
7/7 [==============================] - 0s 3ms/step - loss: 2.2005 - accuracy: 0.2338
Epoch 2/20
7/7 [==============================] - 0s 3ms/step - loss: 1.8832 - accuracy: 0.3988
Epoch 3/20
7/7 [==============================] - 0s 3ms/step - loss: 1.6523 - accuracy: 0.4288
Epoch 4/20
7/7 [==============================] - 0s 3ms/step - loss: 1.4563 - accuracy: 0.4737
Epoch 5/20
7/7 [==============================] - 0s 3ms/step - loss: 1.3148 - accuracy: 0.5475
Epoch 6/20
7/7 [==============================] - 0s 3ms/step - loss: 1.2111 - accuracy: 0.5750
Epoch 7/20
7/7 [==============================] - 0s 3ms/step - loss: 1.1293 - accuracy: 0.6062
Epoch 8/20
7/7 [==============================] - 0s 3ms/step - loss: 1.0536 - accuracy: 0.6288
Epoch 9/20
7/7 [==============================] - 0s 3ms/step - loss: 0.9948 - accuracy: 0.6750
Epoch 10/20
7/7 [==============================] - 0s 4ms/step - loss: 0.9385 - accuracy: 0.6963
Epoch 11/20
7/7 [======================

In [149]:
test_loss, test_acc = model.evaluate(X_test,y_test)

7/7 [==============================] - 0s 1ms/step - loss: 0.9671 - accuracy: 0.6700


In [150]:
print('test_acc: ',test_acc)

test_acc:  0.6700000166893005


Tes accuracy is less than training dataa accuracy. This hints at Overfitting

## Validating our approach
Let's set apart 200 samples in our training data to use as a validation set:

In [151]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

Now let's train our network for 20 epochs:

In [152]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Epoch 1/30
2/2 [==============================] - 0s 77ms/step - loss: 2.2442 - accuracy: 0.2400 - val_loss: 2.0898 - val_accuracy: 0.2700
Epoch 2/30
2/2 [==============================] - 0s 13ms/step - loss: 2.0280 - accuracy: 0.3267 - val_loss: 1.9454 - val_accuracy: 0.2950
Epoch 3/30
2/2 [==============================] - 0s 13ms/step - loss: 1.8476 - accuracy: 0.3333 - val_loss: 1.8531 - val_accuracy: 0.2950
Epoch 4/30
2/2 [==============================] - 0s 13ms/step - loss: 1.7215 - accuracy: 0.3450 - val_loss: 1.7652 - val_accuracy: 0.3250
Epoch 5/30
2/2 [==============================] - 0s 13ms/step - loss: 1.6053 - accuracy: 0.3900 - val_loss: 1.6643 - val_accuracy: 0.3950
Epoch 6/30
2/2 [==============================] - 0s 13ms/step - loss: 1.5046 - accuracy: 0.4567 - val_loss: 1.5861 - val_accuracy: 0.4100
Epoch 7/30
2/2 [==============================] - 0s 14ms/step - loss: 1.4142 - accuracy: 0.4850 - val_loss: 1.4955 - val_accuracy: 0.4200
Epoch 8/30
2/2 [===========

In [153]:
results

[1.0359023809432983, 0.6349999904632568]

## Predictions on Test Data

In [154]:
predictions = model.predict(X_test)

In [155]:
predictions[0].shape

(10,)

In [156]:
np.sum(predictions[0])

1.0

In [157]:
np.argmax(predictions[0])

3

In [158]:
test_pred = model.predict_classes(X_test)
test_proba = model.predict_proba(X_test)
test_proba

Instructions for updating:
Please use `model.predict()` instead.


array([[1.59940362e-04, 3.10898540e-05, 1.01994565e-02, ...,
        4.53037536e-03, 3.38705326e-03, 9.23945159e-02],
       [6.59426689e-01, 1.67315250e-06, 3.32727563e-03, ...,
        2.65943527e-05, 3.95963620e-03, 1.50763258e-01],
       [5.44218346e-03, 5.87247541e-06, 5.13623049e-03, ...,
        5.26007898e-06, 2.15779361e-03, 5.64292133e-01],
       ...,
       [6.37126505e-01, 9.48291345e-05, 9.90096293e-03, ...,
        4.56858725e-05, 4.40746732e-02, 2.54968762e-01],
       [5.92095815e-02, 1.45836854e-08, 1.19431665e-04, ...,
        6.84015902e-07, 2.47452961e-04, 2.12982483e-03],
       [7.15531548e-03, 5.82542678e-04, 2.89253183e-02, ...,
        1.54977562e-02, 1.37430336e-02, 1.15648985e-01]], dtype=float32)

In [161]:
accuracy_score(y_test, test_pred)

0.635